# TODO

## finalize transmission module

In [2]:
import os
import pkg_resources
import tempfile

import leafmap
import whitebox
import rasterio
import yaml

import numpy as np
import geopandas as gpd

from rasterio import features
import rasterio.mask

import cerf

wbt = whitebox.WhiteboxTools()


In [7]:
data_dir = '/Users/d3y010/projects/cerf/data'

states_file = pkg_resources.resource_filename('cerf', 'data/state-abbrev_to_state-name.yml')

original_file = os.path.join(data_dir, 'hifld', 'Electric_Substations-shp', 'Substations.shp')

template_raster = pkg_resources.resource_filename('cerf', 'data/cerf_conus_states_albers_1km.tif')

substation_raster = '/Users/d3y010/Desktop/subs_distance.tif'

natgas_file = os.path.join(data_dir, 'eia', 'NaturalGas_InterIntrastate_Pipelines_US_EIA', 'NaturalGas_Pipelines_US_202001.shp')


In [6]:
from cerf import Interconnection

In [10]:
with rasterio.open(template_raster) as src:
    
    gdf = gpd.read_file(original_file).to_crs(src.crs)


In [11]:
gdf.head()

,OBJECTID,ID,NAME,CITY,STATE,ZIP,TYPE,STATUS,COUNTY,COUNTYFIPS,...,SOURCE,SOURCEDATE,VAL_METHOD,VAL_DATE,LINES,MAX_VOLT,MIN_VOLT,MAX_INFER,MIN_INFER,geometry
0,1,107655,UNKNOWN107655,SHELL LAKE,WI,54871,SUBSTATION,IN SERVICE,WASHBURN,55129,...,http://psc.wi.gov/lmp_zoneinfo/maps/electricmap...,2014-04-07,IMAGERY,2015-08-28,5.0,161.0,69.0,N,Y,POINT (321804.304 931364.975)
1,2,107656,UNKNOWN107656,PRENTICE,WI,54556,SUBSTATION,IN SERVICE,PRICE,55099,...,http://psc.wi.gov/lmp_zoneinfo/maps/electricmap...,2014-04-07,IMAGERY,2015-08-28,3.0,115.0,69.0,N,Y,POINT (444056.365 912112.182)
2,3,107657,UNKNOWN107657,HOLCOMBE,WI,54745,SUBSTATION,IN SERVICE,CHIPPEWA,55017,...,http://psc.wi.gov/lmp_zoneinfo/maps/electricmap...,2014-04-07,IMAGERY,2015-08-28,4.0,115.0,-999999.0,N,N,POINT (382227.045 873605.053)
3,4,107658,SOUTH SHEBOYGAN FALLS,LIMA TOWN OF,WI,53085,SUBSTATION,IN SERVICE,SHEBOYGAN,55117,...,"IMAGERY, OpenStreetMap, http://www.atcllc.com/...",2014-04-07,IMAGERY,2018-09-11,3.0,138.0,69.0,Y,Y,POINT (654201.704 723404.723)
4,5,107659,CHALK HILLS HY,DAGGETT,MI,49821,SUBSTATION,IN SERVICE,MENOMINEE,26109,...,http://www.atcllc.com/wp-content/uploads/2014/...,2014-04-07,IMAGERY,2020-03-23,3.0,138.0,-999999.0,N,N,POINT (639897.480 923692.424)


In [12]:

with open(states_file, 'r') as yml:
    states = yaml.load(yml, Loader=yaml.FullLoader)


gxf = gdf.loc[(gdf['TYPE'] == 'SUBSTATION') &
                       (gdf['STATE'].isin(states.keys())) &
                       (gdf['STATUS'].isin(('IN SERVICE', 'UNDER CONST')))].copy()


In [13]:
gdf = gxf.copy()

gdf['_rval_'] = 0
gdf['_rval_'] = np.where((gdf['MIN_VOLT'] >= -99999999) & (gdf['MIN_VOLT'] < 100), 678, gdf['_rval_'])
gdf['_rval_'] = np.where((gdf['MIN_VOLT'] >= 100) & (gdf['MIN_VOLT'] < 120), 756, gdf['_rval_'])
gdf['_rval_'] = np.where((gdf['MIN_VOLT'] >= 120) & (gdf['MIN_VOLT'] < 150), 791, gdf['_rval_'])
gdf['_rval_'] = np.where((gdf['MIN_VOLT'] >= 150) & (gdf['MIN_VOLT'] < 170), 808, gdf['_rval_'])
gdf['_rval_'] = np.where((gdf['MIN_VOLT'] >= 170) & (gdf['MIN_VOLT'] < 300), 863, gdf['_rval_'])
gdf['_rval_'] = np.where((gdf['MIN_VOLT'] >= 300) & (gdf['MIN_VOLT'] < 400), 1380, gdf['_rval_'])
gdf['_rval_'] = np.where((gdf['MIN_VOLT'] >= 400) & (gdf['MIN_VOLT'] < 99999999), 1458, gdf['_rval_'])


In [15]:


gdf[['_rval_', 'geometry']].to_file('/Users/d3y010/repos/github/cerf/cerf/data/hifld_substations_conus_albers.shp')


In [6]:
transmission_gdf = gdf.copy()

In [7]:
output_rast = '/Users/d3y010/Desktop/rast.tif'
output_dist = '/Users/d3y010/Desktop/dist.tif'
output_alloc = '/Users/d3y010/Desktop/alloc.tif'
output_transmission_cost_file = '/Users/d3y010/Desktop/trans.tif'


In [8]:
transmission_costs = {1: 678, 2: 756, 3: 791, 4: 808, 5: 863, 6: 1380, 7: 1458}


In [ ]:
# instantiate whitebox toolset
wbt = whitebox.WhiteboxTools()

# get the template raster from CERF data
template_raster = pkg_resources.resource_filename('cerf', 'data/cerf_conus_states_albers_1km.tif')



with rasterio.open(template_raster) as src:
    # create 0 where land array
    arr = (src.read(1) * 0).astype(rasterio.float64)

    # update metadata datatype to float64
    metadata = src.meta.copy()
    metadata.update({'dtype': rasterio.float64})

    # reproject transmission data
    gdf = transmission_gdf.to_crs(src.crs)

    # get shapes
    shapes = ((geom, value) for geom, value in zip(gdf.geometry, gdf['_rval_']))


with tempfile.TemporaryDirectory() as tempdir:
    
    out_rast = os.path.join(tempdir, 'cerf_transmission_raster.tif')
    out_dist = os.path.join(tempdir, 'cerf_transmission_distance.tif')
    out_alloc = os.path.join(tempdir, 'cerf_transmission_allocation.tif')
    out_costs = os.path.join('/Users/d3y010/Desktop', 'cerf_transmission_costs.tif')

    # rasterize transmission vector data and write to memory
    with rasterio.open(out_rast, 'w', **metadata) as dataset:

        # burn features into raster
        burned = features.rasterize(shapes=shapes, fill=0, out=arr, transform=dataset.transform)

        # write the outputs to file
        dataset.write_band(1, burned)

    # calculate Euclidean distance file and write raster; result just stores the return value 0
    dist_result = wbt.euclidean_distance(out_rast, out_dist) #, callback=suppress_callback)

    alloc_result = wbt.euclidean_allocation(out_rast, out_alloc)

    with rasterio.open(out_dist) as dist:
        dist_arr = dist.read(1)

    with rasterio.open(out_alloc) as alloc:
        alloc_arr = alloc.read(1)

    with rasterio.open(out_costs, 'w', **metadata) as out:

        m_to_km_factor = 0.001

        # distance in km * the cost of the nearest substation; outputs $2015/km
        cost_arr = (dist_arr * m_to_km_factor) * alloc_arr

        out.write(cost_arr, 1)


In [ ]:
wbt.euclidean_allocation(output_rast, output_alloc)

In [ ]:
cerf.transmission_to_distance_raster(gdf, substation_raster)


In [34]:
help(cerf.process_eia_natural_gas_pipelines)

AttributeError: module 'cerf' has no attribute 'process_eia_natural_gas_pipelines'

In [4]:
gdf = cerf.process_hifld_substations()


In [4]:
cerf.transmission_to_distance_raster(gdf, substation_raster)


In [26]:
type(target_crs)

rasterio.crs.CRS

In [24]:
target_crs

CRS.from_wkt('PROJCS["USA_Contiguous_Albers_Equal_Area_Conic",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4269"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",37.5],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]')

In [25]:
import pickle

pickle.dump(target_crs, open('/Users/d3y010/repos/github/cerf/cerf/data/crs_usa_contiguous_albers_equal_area_conic.p', 'wb'))

In [12]:
with rasterio.open(template_raster) as src:
    
    target_crs = src.crs
    

    

In [22]:
xdf = gpd.read_file(natgas_file).to_crs(target_crs)

xdf.head()


,TYPEPIPE,Operator,Status,Shape_Leng,geometry
0,Intrastate,Texas Intrastate Pipeline Co,Operating,0.001540,"LINESTRING (135389.070 -873975.050, 135491.564..."
1,Intrastate,Texas Intrastate Pipeline Co,Operating,0.005666,"LINESTRING (134842.306 -874020.789, 135389.070..."
2,Intrastate,Texas Intrastate Pipeline Co,Operating,0.050463,"LINESTRING (137390.391 -880907.052, 139246.403..."
3,Intrastate,Texas Intrastate Pipeline Co,Operating,0.003288,"LINESTRING (139246.403 -886091.229, 139563.340..."
4,Intrastate,Texas Intrastate Pipeline Co,Operating,0.004257,"LINESTRING (139246.403 -886091.229, 138853.103..."


In [23]:
# only keep features with a length > 0
xdf = xdf.loc[xdf.geometry.length > 0].copy()



In [24]:
xdf = xdf.loc[xdf['Status'] == 'Operating'].copy()


xdf['_rval_'] = 737

In [25]:
xdf[['_rval_', 'geometry']].to_file('/Users/d3y010/projects/cerf/data/eia_natural_gas_pipelines_conus_albers.shp')
                          

In [11]:
with rasterio.open(template_raster) as src:
    target_crs = src.crs


In [13]:
with open(states_file, 'r') as yml:
    states = yaml.load(yml, Loader=yaml.FullLoader)


In [14]:
gdf = gpd.read_file(original_file)


In [15]:
# keep only substations in the CONUS that are either in service or under construction
gdf = gdf.loc[(gdf['TYPE'] == 'SUBSTATION') & 
              (gdf['STATE'].isin(states.keys())) &
              (gdf['STATUS'].isin(('IN SERVICE', 'UNDER CONST')))].copy()

# set field to be used as raster value
gdf['value'] = 1


In [19]:
xdf = gdf[['geometry']].copy()
xdf = xdf.to_crs(target_crs)


xdf.to_file('/Users/d3y010/projects/cerf/data/hifld_substations_conus_albers.shp')


In [16]:
with rasterio.open(template_raster) as src:
    
    # create 0 where land array
    arr = src.read(1) * 0
    
    metadata = src.meta.copy()
    target_crs = src.crs
    
    with rasterio.open(substation_raster, 'w', **metadata) as out:

        # reproject transmission data
        gdf = gdf.to_crs(target_crs)

        # get shapes
        shapes = ((geom, value) for geom, value in zip(gdf.geometry, gdf['value']))

        # burn shapes
        burned = features.rasterize(shapes=shapes, fill=0, out=arr, transform=out.transform)

        out.write_band(1, burned)


In [46]:
from rasterio.io import MemoryFile
from rasterio.plot import show
import tempfile


def suppress_callback(value):
    """Do not log callback output for whitebox functions."""
    
    pass



with tempfile.NamedTemporaryFile(suffix='.tif') as temp:

    with rasterio.open(template_raster) as src:

        # create 0 where land array
        arr = src.read(1) * 0

        metadata = src.meta.copy()
        
        # reproject transmission data
        gdf = gdf.to_crs(src.crs)

        # get shapes
        shapes = ((geom, value) for geom, value in zip(gdf.geometry, gdf['_rval_']))

    # rasterize transmission vector data and save to memory
    with rasterio.open(temp.name, 'w', **metadata) as dataset:

        # burn features into raster
        burned = features.rasterize(shapes=shapes, fill=0, out=arr, transform=dataset.transform)

        # write the outputs to file
        dataset.write_band(1, burned)

    # calculate Euclidean distance file and write raster; result just stores the return value 0
    result = wbt.euclidean_distance(temp.name, substation_raster, callback=suppress_callback)



In [30]:
dataset.

['/vsimem/b8cb4085-dcb9-4f53-bbd4-2171a4a98df5/b8cb4085-dcb9-4f53-bbd4-2171a4a98df5.tif']

In [22]:
t = tempfile.NamedTemporaryFile()


In [23]:
print(t.name)
t.close()

/var/folders/1q/kbqf0s_90m907n320mhb_gq46y9fxr/T/tmpnv4hwuep


In [25]:

def suppress_callback(value):
    """Do not log callback output for whitebox functions."""
    
    pass

# value of v will be 0 if succ
v = wbt.euclidean_distance(
    substation_raster, 
    '/Users/d3y010/Desktop/euclidean_distance.tif',
    callback=suppress_callback
)


In [ ]:
output_units='km'

    :param output_units:                        Units of distance. Default is kilometers (km).  Alternate options
                                                are miles (mile) and meters (m).
    :type output_units:                         str
            
            
# get conversion factor for native units of meters
units_lowercase = output_units.lower()
if units_lowercase == 'km':
    conversion_factor = 0.001
elif units_lowercase == 'mile':
    conversion_factor = 0.000621371
elif units_lowercase == 'm':
    conversion_factor = 1
else:
    raise KeyError(f"'output_units' must be in 'km' or 'mile'. Option '{output_units}' not supported.")


## finalize lmp module

## finalize read the docs

## regenerate figure for paper

## metarepo

## finalize tutorial

## cerf on pypy

In [1]:
import os

import rasterio


In [3]:
sdf = gpd.read_file('/Users/d3y010/projects/cerf/data/eia/shp/cerf_substations_clean.shp')

In [4]:
sdf.head()

,NAME,TYPE,STATUS,MAX_VOLT,MIN_VOLT,geometry
0,UNKNOWN107655,SUBSTATION,IN SERVICE,161.0,69.0,POINT (-10226336.565 5743316.246)
1,UNKNOWN107656,SUBSTATION,IN SERVICE,115.0,69.0,POINT (-10053464.958 5706699.742)
2,UNKNOWN107657,SUBSTATION,IN SERVICE,115.0,-999999.0,POINT (-10144276.066 5656784.281)
3,SOUTH SHEBOYGAN FALLS,SUBSTATION,IN SERVICE,138.0,69.0,POINT (-9778125.514 5420826.129)
4,CHALK HILLS HY,SUBSTATION,IN SERVICE,138.0,-999999.0,POINT (-9773953.789 5702771.663)
